<a href="https://colab.research.google.com/github/PreranaCK/Detecting-Cyber-Security-Using-Deep-Learning/blob/main/SaaS_fintech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy


In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Parameters
n_users = 10000
user_ids = [f"user_{i}" for i in range(1, n_users + 1)]
signup_dates = pd.date_range(start="2023-01-01", periods=n_users, freq="T")
device_types = np.random.choice(['Desktop', 'Mobile', 'Tablet'], size=n_users, p=[0.5, 0.4, 0.1])
regions = np.random.choice(['North America', 'Europe', 'Asia', 'South America'], size=n_users, p=[0.4, 0.3, 0.2, 0.1])

# Funnel conversion probabilities
verify_prob = 0.9
onboard_prob = 0.7
initiate_prob = 0.5
execute_prob = 0.3

# Revenue weights
weights = {'verify': 0.1, 'onboard': 0.2, 'initiate': 0.3, 'execute': 0.4}

# Funnel completion simulation
verify_complete = np.random.binomial(1, verify_prob, n_users)
onboard_complete = [np.random.binomial(1, onboard_prob) if v == 1 else 0 for v in verify_complete]
initiate_complete = [np.random.binomial(1, initiate_prob) if o == 1 else 0 for o in onboard_complete]
execute_complete = [np.random.binomial(1, execute_prob) if i == 1 else 0 for i in initiate_complete]

# Revenue calculation
revenue_generated = [
    round(
        v * weights['verify'] +
        o * weights['onboard'] +
        i * weights['initiate'] +
        e * weights['execute'], 2
    )
    for v, o, i, e in zip(verify_complete, onboard_complete, initiate_complete, execute_complete)
]

# Final DataFrame
df = pd.DataFrame({
    'user_id': user_ids,
    'signup_date': signup_dates,
    'device_type': device_types,
    'region': regions,
    'verify_complete': verify_complete,
    'onboard_complete': onboard_complete,
    'initiate_complete': initiate_complete,
    'execute_complete': execute_complete,
    'revenue_generated': revenue_generated
})

df.to_csv("saas_funnel_data.csv", index=False)


/tmp/ipython-input-1692835014.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  signup_dates = pd.date_range(start="2023-01-01", periods=n_users, freq="T")


In [ ]:
stage_completion = df[['verify_complete', 'onboard_complete', 'initiate_complete', 'execute_complete']].mean()
print("Stage Completion Rates (%):")
print(stage_completion * 100)


Stage Completion Rates (%):
verify_complete      90.12
onboard_complete     63.33
initiate_complete    31.26
execute_complete      9.37
dtype: float64


In [ ]:
def simulate_lift(df, stage, lift_percent=0.01):
    df_copy = df.copy()
    improved = df_copy[df_copy[stage] == 0].sample(frac=lift_percent)
    df_copy.loc[improved.index, stage] = 1

    # Recalculate dependent stages
    if stage == 'verify_complete':
        df_copy['onboard_complete'] = df_copy['verify_complete'].apply(lambda x: np.random.binomial(1, onboard_prob) if x == 1 else 0)
    if stage == 'onboard_complete':
        df_copy['initiate_complete'] = df_copy['onboard_complete'].apply(lambda x: np.random.binomial(1, initiate_prob) if x == 1 else 0)
    if stage == 'initiate_complete':
        df_copy['execute_complete'] = df_copy['initiate_complete'].apply(lambda x: np.random.binomial(1, execute_prob) if x == 1 else 0)

    # Recalculate revenue
    df_copy['revenue_generated'] = [
        round(
            v * weights['verify'] +
            o * weights['onboard'] +
            i * weights['initiate'] +
            e * weights['execute'], 2
        )
        for v, o, i, e in zip(
            df_copy['verify_complete'],
            df_copy['onboard_complete'],
            df_copy['initiate_complete'],
            df_copy['execute_complete']
        )
    ]

    return df_copy['revenue_generated'].sum()

# Original revenue
original_revenue = df['revenue_generated'].sum()

# Simulate lift for each stage
for stage in ['verify_complete', 'onboard_complete', 'initiate_complete', 'execute_complete']:
    new_revenue = simulate_lift(df, stage)
    lift = new_revenue - original_revenue
    print(f"Revenue lift from 1% improvement in {stage}: ₹{round(lift, 2)}")


Revenue lift from 1% improvement in verify_complete: ₹-3.4
Revenue lift from 1% improvement in onboard_complete: ₹20.6
Revenue lift from 1% improvement in initiate_complete: ₹23.1
Revenue lift from 1% improvement in execute_complete: ₹36.4


In [ ]:
df.to_csv("saas_funnel_data.csv", index=False)


In [ ]:
from google.colab import files
files.download("saas_funnel_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>